In [12]:
!pip install "cloud-sql-python-connector[pymysql]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.38.0 which is incompatible.


In [13]:
from google.cloud import aiplatform
from vertexai.preview.language_models import TextEmbeddingModel
import tqdm,time
from google.cloud.sql.connector import Connector
import sqlalchemy

In [2]:
PROJECT_ID = "project id"
LOCATION = "project-location"

aiplatform.init(project=PROJECT_ID,location=LOCATION)
model = TextEmbeddingModel.from_pretrained("text-embedding-005")

def get_embeddings_wrapper(texts, batch_size=5):
    embeddings = []
    for i in tqdm.tqdm(range(0, len(texts), batch_size)):
        time.sleep(1)
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.get_embeddings(batch_texts)
        embeddings.extend([embedding.values for embedding in batch_embeddings])
    return embeddings


In [14]:
def getconn():
    connector = Connector()

    conn= connector.connect(
        "connection name",
        "pymysql",
        user="user",
        password="password",
        db="db name"
    )
    return conn


def get_pool():
    pool = sqlalchemy.create_engine(
       "mysql+pymysql://",
        creator=getconn,
    )
    return pool

In [28]:
def getItem(n):
  pool = get_pool()
  with pool.connect() as db_conn:
    select_query = sqlalchemy.text(f"""SELECT *
                    FROM home_run_data
                    LIMIT 1 OFFSET {n};""")
    result = db_conn.execute(select_query)
    users = result.fetchone()
    return users


In [3]:
candidates_endpoint = aiplatform.MatchingEngineIndexEndpoint("index endpoint")


In [45]:
user_prompt = "Let me watch some exiting matches"

input_embeddings = get_embeddings_wrapper([user_prompt])

DEPLOYED_INDEX_ID = f"deployed_index"

response = candidates_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=input_embeddings,
    num_neighbors=5,
)

100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


In [46]:
for item in response[0]:
  print(getItem(item.id))

('7dba1a4c-31e7-4772-bde2-2439968c2280', 'Joc Pederson homers (25) on a fly ball to right field.    Andrew Toles scores.', 106.8, 398.0, 23.0, 'https://sporty-clips.mlb.com/RzY4OUxfWGw0TUFRPT1fQVZVREJWVlZVbEVBQzFOWFVRQUFCZ0ZVQUFOUkJsTUFBRkVHQ1FvRENWQlhBZ3BX.mp4\r')
('3b9ecf94-e3fe-4d79-8b33-efe174dfa53e', 'Marwin Gonzalez homers (13) on a line drive to right field.', 106.6, 390.0, 22.0, 'https://sporty-clips.mlb.com/Mk9vbFBfWGw0TUFRPT1fQlZOWUFRZFNYd1lBWFZGVUF3QUFWUUZmQUZsWFVGSUFWd05RQVFRQlZWVUVCUUVE.mp4\r')
('279c99a0-9137-4400-b584-5cff70b4003e', 'Justin Morneau homers (6) on a fly ball to right field.    Jose Abreu scores.', 96.1, 368.0, 37.0, 'https://sporty-clips.mlb.com/OTlxb2pfWGw0TUFRPT1fQkFaWUIxME5Cd0lBQVZNQlVRQUFCUVpXQUFNQVcxVUFCd1pUVmdRRlV3ZFZBQUVE.mp4\r')
('c8265331-1a15-45e8-a0ad-0b84117d0202', 'Curtis Granderson homers (28) on a fly ball to right field.', 98.3, 349.0, 30.0, 'https://sporty-clips.mlb.com/ajlhWFpfWGw0TUFRPT1fVlFsVFVsRUhWUU1BQ1FNRFV3QUFCRk5lQUFBRkFnVUFBZ2NOQk

'1'